In [1]:
// Importing necessary libraries

import org.apache.spark.ml.evaluation.RegressionEvaluator // to evaluvate our model
import org.apache.spark.ml.recommendation.ALS //Alternating least square algorithm to create Recomendation model
import org.apache.spark.ml.tuning.{ParamGridBuilder, CrossValidator} //Library to tune the Parameters
import org.apache.spark.sql.SQLContext //enables to use Sql
import org.apache.spark.sql.functions._ // imports usfull SQL functions
import org.apache.spark.sql.types.DoubleType // a datatype which is used to store long Float values

val sqlContext = new org.apache.spark.sql.SQLContext(sc) //creating a spark context object

import sqlContext.implicits._ // Implicit for converting Scala objects into DataFrames
import sys.process._ //Enables to use shell commands with spark
import org.apache.spark.sql.SparkSession //Importing SparkSession library

import org.apache.spark.mllib.recommendation.Rating

Intitializing Scala interpreter ...

Spark Web UI available at http://DESKTOP-8RE46GS.diatoz.com:4040
SparkContext available as 'sc' (version = 3.0.1, master = local[*], app id = local-1612852754322)
SparkSession available as 'spark'


import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.ml.recommendation.ALS
import org.apache.spark.ml.tuning.{ParamGridBuilder, CrossValidator}
import org.apache.spark.sql.SQLContext
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types.DoubleType
sqlContext: org.apache.spark.sql.SQLContext = org.apache.spark.sql.SQLContext@3fd884f3
import sqlContext.implicits._
import sys.process._
import org.apache.spark.sql.SparkSession


##### DatasetLink

http://files.grouplens.org/datasets/movielens/ml-1m.zip

In [36]:
!dir //shell command to see the list of files in the working directory

Get-ChildItem : A positional parameter cannot be found that accepts argument 'to'.
At line:1 char:1
+ dir //shell command to see the list of files in the working directory
+ ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    + CategoryInfo          : InvalidArgument: (:) [Get-ChildItem], ParameterBindingExcep 
   tion
    + FullyQualifiedErrorId : PositionalParameterNotFound,Microsoft.PowerShell.Commands.G 
   etChildItemCommand
 



In [3]:
val ratings_rdd = sc.textFile("./ml-1m/ratings.dat") //reading a text file using SparkContext

ratings_rdd: org.apache.spark.rdd.RDD[String] = ./ml-1m/ratings.dat MapPartitionsRDD[1] at textFile at <console>:42


In [4]:
println("Number of ratings = " + ratings_rdd.count()) //Counts the number of rows present in the Rdd

Number of ratings = 1000209


In [5]:
ratings_rdd.takeSample(false, 10, 0).foreach(println) // to view sample of dataset

1120::1513::4::974912892
1317::66::1::975212462
5820::953::5::957905663
3804::2148::1::965969627
5046::432::2::962507780
5193::1230::4::961702574
1635::1203::5::976942383
1658::2124::3::974717583
332::1633::5::982552828
5074::3760::3::964468697


In [6]:
// Two options 1- to use built in module or creating an own class
// case class Rating(userId: Int, movieId: Int, rating: Float) //notworking


import org.apache.spark.mllib.recommendation.Rating


In [7]:
val ratings = ratings_rdd.map(x => x.split("::")).map(r => Rating(r(0).toInt, r(1).toInt, r(2).toDouble)).toDF().na.drop()

ratings: org.apache.spark.sql.DataFrame = [user: int, product: int ... 1 more field]


In [8]:
// Diffetrent ways to see the sample of data

ratings.sample(false, 0.0001, seed=0).show(10)

// ratings.take(10).foreach(println)

+----+-------+------+
|user|product|rating|
+----+-------+------+
|  58|   2580|   4.0|
| 136|    235|   5.0|
| 187|    246|   5.0|
| 329|    832|   2.0|
| 368|   1370|   3.0|
| 629|   1013|   4.0|
| 660|   2409|   2.0|
| 662|     19|   3.0|
| 690|   1370|   5.0|
| 852|    480|   3.0|
+----+-------+------+
only showing top 10 rows



In [9]:
// Sample number of ratings per user
val grouped_ratings = ratings.groupBy("user").count().withColumnRenamed("count", "No.of ratings")
grouped_ratings.show(10)

+----+-------------+
|user|No.of ratings|
+----+-------------+
| 148|          624|
| 463|          123|
| 471|          105|
| 496|          119|
| 833|           21|
|1088|         1176|
|1238|           45|
|1342|           92|
|1580|           37|
|1591|          314|
+----+-------------+
only showing top 10 rows



grouped_ratings: org.apache.spark.sql.DataFrame = [user: int, No.of ratings: bigint]


In [10]:
// number of users in the dataset.
println("Number of users = " + grouped_ratings.count())

Number of users = 6040


-----
#### Exploring Movies dataset

In [11]:
// importing dataset
val movies_df = sc.textFile("./ml-1m/movies.dat")

movies_df: org.apache.spark.rdd.RDD[String] = ./ml-1m/movies.dat MapPartitionsRDD[24] at textFile at <console>:44


In [12]:
movies_df.takeSample(false,10,0).foreach(println)

699::To Cross the Rubicon (1991)::Drama
854::Ballad of Narayama, The (Narayama Bushiko) (1958)::Drama
3813::Interiors (1978)::Drama
2505::8MM (1999)::Thriller
3327::Beyond the Mat (2000)::Documentary
3417::Crimson Pirate, The (1952)::Adventure|Comedy|Sci-Fi
1062::Sunchaser, The (1996)::Drama
1080::Monty Python's Life of Brian (1979)::Comedy
197::Stars Fell on Henrietta, The (1995)::Drama
3346::Color Me Blood Red (1965)::Horror


In [13]:
// // Convert movies data to a DataFrame

// case class Movies(movieId: Int, Title: String, Genre: String)

// val movie2 = movies_df.map(x => x.split("::")).map(m =>new Movies(m(0).toInt, m(1).toString, m(2).toString )).toDF()
// movie2.take(5)

In [14]:
// Convert movies data to a DataFrame

val movies = movies_df.map { line =>
      val fields = line.split("::")
      (fields(0).toInt, fields(1).toString, fields(2).toString)      // format: (movieId, movieName, catagory)
    }.toDF("movieId" ,  "Title", "Genre" )

movies: org.apache.spark.sql.DataFrame = [movieId: int, Title: string ... 1 more field]


In [15]:
// val movies_rdd = movies_rdd.withColumnRenamed("_1", "movieId").withColumnRenamed("_2", "Title").withColumnRenamed("_3", "Genre")

movies.show(10)
// movies.take(10).foreach(println)

+-------+--------------------+--------------------+
|movieId|               Title|               Genre|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Animation|Childre...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|        Comedy|Drama|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|Adventure|Children's|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
+-------+--------------------+--------------------+
only showing top 10 rows



In [16]:
println("Number of movies = " + movies.count())

Number of movies = 3883


### Data preprocessing

Split Ratings data into Training (80%) and Test (20%) datasets

In [17]:
// we create two Arrays one for training and other for testing.
val Array(training, test) = ratings.randomSplit(Array(0.8, 0.2), seed=0L)

training: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [user: int, product: int ... 1 more field]
test: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [user: int, product: int ... 1 more field]


In [18]:
// Show resulting Ratings dataset count
var trainingRatio = training.count().toDouble/ratings.count().toDouble*100
var testRatio = test.count().toDouble/ratings.count().toDouble*100

println("Total number of ratings = " + ratings.count())
println("Training dataset count = " + training.count() + ", " + BigDecimal(trainingRatio).setScale(2, BigDecimal.RoundingMode.HALF_UP).toDouble + "%")
println("Test dataset count = " + test.count() + ", " + BigDecimal(testRatio).setScale(2, BigDecimal.RoundingMode.HALF_UP).toDouble+ "%")

Total number of ratings = 1000209
Training dataset count = 799402, 79.92%
Test dataset count = 200807, 20.08%


trainingRatio: Double = 79.92349598933824
testRatio: Double = 20.076504010661772


In [19]:
// training.count().toDouble/ratings.count().toDouble //caleculating percentile

In [20]:
training.sample(false, 0.0001, seed=0).show(10)

+----+-------+------+
|user|product|rating|
+----+-------+------+
|  77|   2907|   1.0|
| 161|    950|   5.0|
| 224|   1569|   3.0|
| 412|   3448|   4.0|
| 466|   1073|   4.0|
| 757|   2642|   2.0|
| 796|   3068|   4.0|
| 801|   1617|   3.0|
| 841|   3916|   4.0|
|1051|   2639|   2.0|
+----+-------+------+
only showing top 10 rows



In [21]:
// BigDecimal(5.0) //creates 128 bit floating point

### Model creation


##### ALS algorithm
Parms of ALS algom
- ratings: ,
- rank: Int,
- numUserBlocks: Int,
- numItemBlocks: Int,
- maxIter: Int,
- regParam: Double,
- implicitPrefs: Boolean,
- alpha: Double,
- nonnegative: Boolean,
- intermediateRDDStorageLevel: ,
- finalRDDStorageLevel: ,
- checkpointInterval: Int,
- seed: Long

In [22]:
val als = new ALS().setMaxIter(10).setRegParam(0.1).setUserCol("user").setItemCol("user").setRatingCol("rating")

als: org.apache.spark.ml.recommendation.ALS = als_0c23a02c8180


In [23]:
val model = als.fit(training)

model: org.apache.spark.ml.recommendation.ALSModel = ALSModel: uid=als_0c23a02c8180, rank=10


In [24]:
// println(als.explainParams)

In [25]:
val predictions = model.transform(test).na.drop()

predictions: org.apache.spark.sql.DataFrame = [user: int, product: int ... 2 more fields]


In [26]:
predictions.distinct().show(10)

+----+-------+------+----------+
|user|product|rating|prediction|
+----+-------+------+----------+
|1591|   3107|   5.0| 4.2886376|
|1645|   3513|   1.0| 3.9887228|
|1645|   3869|   5.0| 3.9887228|
|2142|    589|   5.0|  3.485135|
|3997|    429|   2.0|  3.474543|
|5300|   1261|   4.0| 4.1530066|
|5300|   1294|   4.0| 4.1530066|
| 243|   1968|   3.0| 3.3926785|
| 243|   2858|   5.0| 3.3926785|
| 392|    110|   5.0| 3.4472303|
+----+-------+------+----------+
only showing top 10 rows



### Evaluating the model using RMSE

In [27]:
val evaluator = new RegressionEvaluator().setMetricName("rmse").setLabelCol("rating").setPredictionCol("prediction")
val rmse = evaluator.evaluate(predictions)
println("Root-mean-square error = " + rmse)

Root-mean-square error = 1.0347301874121835


evaluator: org.apache.spark.ml.evaluation.RegressionEvaluator = RegressionEvaluator: uid=regEval_41ecae40b541, metricName=rmse, throughOrigin=false
rmse: Double = 1.0347301874121835


In [28]:
// evaluator.isLargerBetter

#### Tune the Model

In [29]:
// Building a Parameter Grid specifying what parameters and values will be evaluated in order to determine the best combination.
val paramGrid = new ParamGridBuilder().addGrid(als.regParam, Array(0.01, 0.1)).build()

paramGrid: Array[org.apache.spark.ml.param.ParamMap] =
Array({
	als_0c23a02c8180-regParam: 0.01
}, {
	als_0c23a02c8180-regParam: 0.1
})


In [30]:
// cross validation with the defined parameter grid
val cv = new CrossValidator().setEstimator(als).setEvaluator(evaluator).setEstimatorParamMaps(paramGrid).setNumFolds(2)

cv: org.apache.spark.ml.tuning.CrossValidator = cv_3ca9eb6a1284


In [31]:
// Validating the parameter grid values
cv.getEstimatorParamMaps.foreach(println)

{
	als_0c23a02c8180-regParam: 0.01
}
{
	als_0c23a02c8180-regParam: 0.1
}


In [32]:
// Cross-evaluate to find the best model
val cvModel = cv.fit(training) // cross validation

cvModel: org.apache.spark.ml.tuning.CrossValidatorModel = CrossValidatorModel: uid=cv_3ca9eb6a1284, bestModel=ALSModel: uid=als_0c23a02c8180, rank=10, numFolds=2


In [33]:
println("Best fit root-mean-square error = " + evaluator.evaluate(cvModel.transform(test).na.drop()))

Best fit root-mean-square error = 1.032061455952174


#### recommend movies to a user

In [34]:
val userId = 3000

userId: Int = 3000


In [35]:
val movies_watched = ratings.filter(ratings("user") === userId)
movies_watched.show(10)

+----+-------+------+
|user|product|rating|
+----+-------+------+
|3000|   2987|   4.0|
|3000|   2990|   3.0|
|3000|   3793|   3.0|
|3000|   1252|   4.0|
|3000|   2997|   4.0|
|3000|   1259|   3.0|
|3000|    589|   4.0|
|3000|      9|   1.0|
|3000|   1265|   5.0|
|3000|    733|   5.0|
+----+-------+------+
only showing top 10 rows



movies_watched: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [user: int, product: int ... 1 more field]


#### Future to_do List
- Show user 3000's top 10 rated movies (with movie title, genre, and rating)
- Determining what movies user 3000 has not already watched and rated so that we can make new movie recommendations


In [53]:
"foo3".map(_.toString)

res28: scala.collection.immutable.IndexedSeq[String] = Vector(f, o, o, 3)


In [38]:

import vegas._
import vegas.render.WindowRenderer._

<console>: 42: error: not found: value vegas

In [37]:
%AddDeps org.vegas-viz vegas_2.11 {vegas-version} --transitive

<console>: 44: error: not found: value %

In [36]:
import $ivy.`org.vegas-viz::vegas:0.3.0`

<console>: 42: error: not found: value $ivy